In [ ]:
import numpy as np

# Outils sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Outils pytorch
import torch # package Racine
from torch import nn # Réseau de Neurones (Neural Network)
import torch.nn.functional as F # Couches, fonctions d'activation ...
import torch.autograd as autograd # Calcul dérivée (Gradient)
import torch.optim as optim # Optimiser pour la descente de Gradient

# Librairie graphique plotly

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

## Construction d'une classe module

In [ ]:
class ClasseModele(nn.Module):
    
    def __init__(self, input_dim):
        super(ClasseModele, self).__init__()        
        self.layer1 = nn.Linear(input_dim,50) # Première couche de 50 neurones
        self.layer2 = nn.Linear(50, 20)       # deuxième couche de 20 neurones        
        self.layer3 = nn.Linear(20, 3)        # Couche de sortie de 3 neurones (3 classes)
        
    def forward(self, x):
        # https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))        
        x = F.softmax(self.layer3(x),dim=1) # Application de la fonction sofmax à la couche de sortie
        return x    

## Chargement des données Iris

In [ ]:
features, labels = load_iris(return_X_y=True)
print(features.shape,labels.shape)
iris = np.concatenate((features, labels.reshape([150,1])), axis=1)
iris[np.random.randint(len(iris), size=10)]

## Création d'un set de train et d'un set de test (dev)

In [ ]:
features_train,features_dev, labels_train, labels_dev = train_test_split(features, labels, random_state=42)
print(features_train.shape,features_dev.shape, labels_train.shape, labels_dev.shape)
labels_dev[:10]

## Apprentissage avec PyTorch

In [ ]:
# Création d'un objet Modèle
model = ClasseModele(features_train.shape[1])

# choix de l'algorithme de Descente de Gradient et du learning Rate
# https://pytorch.org/docs/stable/optim.html#algorithms
optimizer = optim.Adam(model.parameters(), lr=0.01)

# choix de la fonction de coût
# https://pytorch.org/docs/stable/nn.html#loss-functions
loss_fn = nn.CrossEntropyLoss()

#Nombre d'itération sur les données
epochs = 100

print(model)

In [ ]:
x_train, y_train = torch.from_numpy(features_train).float(), torch.from_numpy(labels_train).long()
for epoch in range(1, epochs+1):    
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    if not epoch%10 :        
        print('Epoch #%i Loss=%.4f'%(epoch,loss))               
    
    optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    optimizer.step()      # Met à jours les paramètres du réseau

## Prédiction

In [ ]:
#Création du Tensor x_dev, de type float
x_dev = torch.from_numpy(features_dev).float()
#Exécution du modèle
pred = model(x_dev)
#Sortie pred du computation graph, et conversion en tableau numpy
print(pred.grad_fn)
pred = pred.detach()
print(pred.grad_fn)
pred = pred.numpy()
pred

## Calcul du score de précision

### Transformation des probabilités en résultat unique

In [ ]:
prevision_type_iris =  np.argmax(model(x_dev).detach().numpy(),axis=1)
prevision_type_iris

### Calcul de la précision

In [ ]:
print ("Précision de la prévision = %.1f%% "%(accuracy_score(labels_dev, prevision_type_iris)*100))
print('%i prévision(s) correcte(s), %i erreur(s)'%(sum(prevision_type_iris==labels_dev),sum(prevision_type_iris!=labels_dev)))

### Comparaison Courbes 

In [ ]:
acc_train=[]
acc_dev=[]
tab_epoch=[]
new_model = ClasseModele(features_train.shape[1])
new_optimizer = optim.Adam(new_model.parameters(), lr=0.01)
for epoch in range(1, epochs+1):    
    y_pred = new_model(x_train)
    loss = loss_fn(y_pred, y_train)
    
    if not epoch%10 :                
        print('Epoch #%i Loss=%.2f'%(epoch,loss.item()))        
        tab_epoch.append(epoch)
        acc_dev.append(
            accuracy_score(
                labels_dev,
                np.argmax(new_model(x_dev).detach().numpy(),axis=1)
            )*100            
        )
        acc_train.append(
            accuracy_score(
                labels_train,
                np.argmax(new_model(x_train).detach().numpy(),axis=1)
            )*100
        )           
    new_optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    new_optimizer.step()      # Met à jours les paramètres du réseau

print(acc_train,acc_dev)

## Graphique

In [ ]:
train_line =go.Scatter(x=tab_epoch,y=acc_train,name='Train set')
dev_line =go.Scatter(x=tab_epoch,y=acc_dev,name='Dev set')

layout = go.Layout(title="Comparaison Précision Train/Test",titlefont=dict(size=40),autosize=False, width=1100,height=1100)

data=[train_line,dev_line]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# import plotly.graph_objects as go
# import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=x, y=x**2))
fig.show()